#Disclaimer to the user
 the prompt travel generation doesn't work in certain colab server locations, so please disconnect, and reconnect the runtime to change the runtime location.

 you can check the server location using the command: !curl ipinfo.io

 Please check locations in the below link https://developers.generativeai.google/available_regions

In [ ]:
# original author: tech-practice ;  original github repo: https://github.com/s9roll7/animatediff-cli-prompt-travel

In [ ]:
# Use this to check the location of the server
!curl ipinfo.io

{
  "ip": "34.83.121.56",
  "hostname": "56.121.83.34.bc.googleusercontent.com",
  "city": "The Dalles",
  "region": "Oregon",
  "country": "US",
  "loc": "45.5946,-121.1787",
  "org": "AS396982 Google LLC",
  "postal": "97058",
  "timezone": "America/Los_Angeles",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
# it will ask you for permission to access the google drive folder,
# please make sure that you are accessing the drive through the same account that you are running the colab notebook with.

!apt-get -qq install aria2

from google.colab import drive
import os, shutil, gc, time
import gdown
base_path = "/content/animatediff-cli-prompt-travel"
pwd = %pwd
print(pwd)
if pwd != base_path:
  !git clone https://github.com/s9roll7/animatediff-cli-prompt-travel
  %cd {base_path}

  # Mount google drive
  drive.mount("GoogleDrive")
  gd_path = "GoogleDrive/MyDrive"
  gd_output_dir = gd_path + "/AnimateDiffPromptTravel"+"/output"

  # create a folder called "AnimateDiffPromptTravel" under "GoogleDrive/MyDrive"
  os.makedirs(gd_output_dir, exist_ok=True)

  #Install all dependencies under the base path
  !pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
  !pip install .[dev]
  !pip install xformers mediapipe
  !pip install onnxruntime-gpu pandas
  from IPython.display import clear_output
  clear_output()

In [ ]:
# AnimateDiff MotionModule download, you can customize this in the last code cell
def dnld_mm(mm):
  data_dir = "data"
  motion_module = mm
  motion_module_path = f"models/motion-module/{motion_module}.ckpt"
  motion_module_file_path = os.path.join(data_dir, motion_module_path)
  if not os.path.exists(motion_module_file_path):
    url = f"https://huggingface.co/guoyww/animatediff/resolve/main/{motion_module}.ckpt"
    !aria2c {url} -o {motion_module_file_path} #Download the MM model checkpoint file from the URL
  return motion_module_path #Vikram 12/1/23

In [ ]:
# Standard Diffussion model download
# feel free to experiment other models, you can customize this in the last code cell
def dnld_sd(url, name):
  data_dir = "data"
  model_url = url
  model_name = name
#   model_path = f"models/sd/{model_name}.safetensors" #Vikram 12/1/23
  model_path = f"models/sd/{model_name}" #Vikram 12/1/23
  model_file_path = os.path.join(data_dir, model_path)
  if not os.path.exists(model_file_path):
    !aria2c {model_url} -o {model_file_path} #Download the MM model checkpoint file from the URL

  return model_path #Vikram 12/1/23

In [ ]:
# Colab free version has limited RAM. SO, delete the images from controlnet folder so that the program runs without interruptions.
def cleanup(folder_path):
  if len(os.listdir(folder_path)) > 1:
    os.remove(f"{folder_path}/0000.png")
    os.remove(f"{folder_path}/0016.png")
    os.remove(f"{folder_path}/0032.png")

In [ ]:
# take about 14-15 minutes depending on the diffussion model that you chose, and the total number of frames that are geenrated.
def inference(config, W, H, L, C):
  !animatediff generate -c {config} -W {W} -H {H} -L {L} -C {C}

In [ ]:
# Copy generated contents to Google drive
def cpy_to_drive():
  output_dir = "output"
  gc.collect()

  def get_new_dir(parent_dir):
    sub_dirs = []
    for path in os.listdir(parent_dir):
      dir_path = os.path.join(parent_dir, path)
      if os.path.isdir(dir_path):
        sub_dirs.append(dir_path)
    # print("VIKRAM VIKRAM VIKRAM", sub_dirs)
    return max(sub_dirs, key = os.path.getmtime)

  result_dir = get_new_dir(output_dir)
  dest_dir = os.path.join(gd_output_dir, os.path.basename(result_dir))
  shutil.copytree(result_dir, dest_dir, dirs_exist_ok = True) #Copy the file directory tree to the google drive folder
  print(f"🎉 The output files have been copied into {dest_dir}. (in your connected Google Drive) See you there! 🎉")

In [ ]:
def json_modif(config, L, mmpath, sdpath): #Vikram 12/1/23

  #BOC 11/28
  !pip install google-generativeai
  import google.generativeai as palm
  palm.configure(api_key='')
  #EOC 11/28

  #------------------------------------------------
  models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

  model = models[0].name

  no_lines = "5" #@param {allow-input: true, type: "string"}
  fps = 16 #@param {allow-input: true, type: "integer"}
  if fps > L:
    raise ValueError("The given FPS is more than the total number of frames generated. Please re-run the cell, and give proper inputs.")


  #Vikram 12/1/23
  op_format = "mp4" #@param ["mp4", "gif", "webm"]

  head_prompt = "(standing,full_body), 5men on the left, 5men on the right, road in the middle with people, blue_sky, town" #@param {allow-input: true, type: "string"}
  prompt_travel = "(standing,full_body), 5men_on_the_left, guns_in_hands, 5men_on_right, guns_in_hands, road_in_the_middle_with_people_walking, angry, tention, blue_sky, town" #@param {allow-input: true, type: "string"}
  tail_prompt = "face, looking at the viewer, detailed background" #@param {allow-input: true, type: "string"}
  n_prompt = ["worst quality", "low quality:1.4", "blurry faces"] #@param {allow-input: true, type: "raw"}

  prompt = f"""
  You are an expert at text genetation.

  expand the prompt into {no_lines} lines story should have the coherence from one line to the next line like a video

  Prompt: {prompt_travel}

  """
  #Generation of the prompt travel
  completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0.1,
    # The maximum length of the response
    max_output_tokens=800,
  )


  #split the prompt travel into separate sentences

  prompt_map = completion.result.split(".")

  prompt_map1 = []
  for i in range(len(prompt_map)):
    if i%2 != 0:
      prompt_map1.append(prompt_map[i])



  #overwrite the existing data in the json file
  import json
  # Read the existing JSON file
  with open(config, 'r') as json_file:
      data = json.load(json_file)
  i = 0

  #path to the sd model
  data["path"] = sdpath #Vikram 12/1/23
  #path to the mm model
  data["motion_module"] = mmpath #Vikram 12/1/23

  #head prompt
  data['head_prompt'] = head_prompt

  #prompt travel
  for line in prompt_map1:
    pos = str(i)
    data['prompt_map'][pos] = line
    i+=fps

  #tail prompt
  data['tail_prompt'] = tail_prompt

  #negative prompt
  data['n_prompt'] = n_prompt

  #-----------------------------------------------
  #Output details
  data['output']['format'] = op_format
  data['output']['fps'] = fps

  # Write the updated data back to the JSON file
  with open(config, 'w') as json_file:
      json.dump(data, json_file, indent=2)

  return config, prompt_map1
  #------------------------------------------------

In [ ]:
# displaying generated prompt
display_prompt = True #@param {type:"boolean"}
generate_prompt = True #@param {type:"boolean"}

# -- Parameters for the program to run --

# Please make sure that you have inputs for the prompt to travel throughout the frames.
# if you want to generate video with 128 frames with 8FPS, then write the prompt accordingly.
# The guideline for the prompt can be seen directly in the original repo

# Motion Module
mm = "mm_sd_v15_v2" #@param {allow-input: true, type: "string"}

# Standard Diffussion model
url = "https://civitai.com/api/download/models/102222" #@param {allow-input: true, type: "string"}
name = "xxmix9realistic_v40.safetensors" #@param {allow-input: true, type: "string"}

# File path to fetch the prompt for the application
config = "GoogleDrive/MyDrive/AnimateDiffPrompts/prompt_dummy.json"#@param {allow-input: true, type: "string"}

# Width of the frame
W = 80 #@param {allow-input: true, type: "number"}
# Height of the frame
H = 80 #@param {allow-input: true, type: "number"}
# Total number of frames
L = 80 #@param {allow-input: true, type: "number"}

C = 16 #@param {allow-input: true, type: "number"}


mmpath = dnld_mm(mm) #Vikram 12/1/23
sdpath = dnld_sd(url, name) #Vikram 12/1/23

if generate_prompt:
    # editing the config file (prompt file)
    config, prompt_map1 = json_modif(config, L, mmpath, sdpath) #Vikram 12/1/23

    # Check the value of the checkbox
    if display_prompt:
        # from ipywidgets import Output
        from IPython.display import clear_output
        clear_output()
        print(prompt_map1)
        # Add actions to perform when the checkbox is checked
    else:
        print("Display checkbox is unchecked.")
        # Add actions to perform when the checkbox is unchecked


# Clean up of the control net reference image folder
folder_path = "data/controlnet_image/test/controlnet_softedge"
cleanup(folder_path)
folder_path = "data/controlnet_image/test/controlnet_openpose"
cleanup(folder_path)

inference(config, W, H, L, C)

cpy_to_drive()
